In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

In [51]:
data = pd.read_csv('train.csv')

In [52]:
data.head(25)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644
5,12915,59,3.0,80.384479,46.683720,12.0,5,17.0,2011,0.309479,B,B,35,7715,4,990.0,0,6,B,215898.447742
6,14549,154,2.0,62.254114,37.160377,7.0,3,5.0,1960,0.460556,B,B,20,4386,14,NaN,1,5,B,296021.204377
7,11993,74,2.0,80.312926,NaN,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,221244.156664
8,5172,1,2.0,64.511437,NaN,1.0,9,17.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,229102.795999
9,8649,23,1.0,46.461409,18.915552,8.0,13,17.0,2014,0.075779,B,B,6,1437,3,NaN,0,2,B,95380.220993


In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [54]:
data.loc[data['HouseYear']==20052011, 'HouseYear'] = 2008
data.loc[data['HouseYear']==4968, 'HouseYear'] = 1968

In [55]:
data['HouseYear'].unique()

array([1969, 1978, 1968, 1977, 1976, 2011, 1960, 2014, 1973, 1959, 1999,
       1980, 1979, 1983, 2001, 2012, 2002, 1996, 1964, 2018, 1972, 1965,
       1984, 1961, 1971, 1963, 2017, 1970, 1981, 2003, 2016, 1991, 1975,
       2006, 2009, 1985, 1974, 1994, 2000, 1987, 1998, 2005, 1990, 1982,
       1997, 2015, 2008, 2010, 2004, 2007, 1967, 1957, 1962, 1993, 1966,
       1955, 1937, 1992, 1954, 1995, 2019, 1948, 1986, 2013, 1989, 1958,
       1938, 1956, 1988, 2020, 1951, 1952, 1935, 1914, 1932, 1950, 1917,
       1918, 1940, 1942, 1939, 1934, 1931, 1919, 1912, 1953, 1936, 1947,
       1929, 1930, 1933, 1941, 1916, 1910, 1928])

In [56]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,1984.86600,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,18.41187,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1910.00000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1974.00000,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1977.00000,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2001.00000,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2020.00000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [57]:
data.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
       'Helthcare_2', 'Shops_1', 'Shops_2', 'Price'],
      dtype='object')

In [58]:
print(data['Ecology_1'].nunique())
print(data['Ecology_2'].nunique())
print(data['Ecology_3'].nunique())
print(data['Healthcare_1'].nunique())
print(data['Social_1'].nunique())
print(data['Social_2'].nunique())
print(data['Social_3'].nunique())
print(data['Shops_1'].nunique())
print(data['Shops_2'].nunique())

129
2
2
79
51
142
30
16
2


In [59]:
data['EcoBA'] = data['Ecology_3'] + data['Ecology_2']
data['EcoBA'].unique()

array(['BB', 'BA', 'AB'], dtype=object)

In [60]:
data.loc[:150, ['DistrictId', 'Square', 'LifeSquare', 'KitchenSquare',
       'Ecology_1', 'Ecology_2', 'EcoBA',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
       'Helthcare_2', 'Shops_1', 'Shops_2', 'Price']].head()

,DistrictId,Square,LifeSquare,KitchenSquare,Ecology_1,Ecology_2,EcoBA,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,35,47.981561,29.442751,6.0,0.089040,B,BB,B,33,7976,5,NaN,0,11,B,184966.930730
1,41,65.683640,40.049543,8.0,0.000070,B,BB,B,46,10309,1,240.0,1,16,B,300009.450063
2,53,44.947953,29.197612,0.0,0.049637,B,BB,B,34,7759,0,229.0,1,3,B,220925.908524
3,58,53.352981,52.731512,9.0,0.437885,B,BB,B,23,5735,3,1084.0,0,5,B,175616.227217
4,99,39.649192,23.776169,7.0,0.012339,B,BB,B,35,5776,1,2078.0,2,4,B,150226.531644


In [61]:
data.groupby(['EcoBA',], sort=True)[['Healthcare_1']].mean()

,Healthcare_1
EcoBA,
AB,1638.865455
BA,1377.979381
BB,1109.945549


In [62]:
data.groupby(['EcoBA','Shops_2'], sort=True)[['Price', 'Rooms', 'Shops_1', 'Healthcare_1']].mean()

Price     Rooms   Shops_1  Healthcare_1
EcoBA Shops_2                                                 
AB    B        243179.560704  1.920000  4.178182   1638.865455
BA    A        159386.651546  1.642857  0.000000    370.000000
      B        198833.005156  1.951807  2.000000   1548.000000
BB    A        192908.557092  1.895191  2.036991   1420.142857
      B        215416.899449  1.888965  4.462516   1100.685928

Средние социальные показатели меньше в категории A чем у B (Shops_2), как и стоимость квартир

In [63]:
data.groupby(['Shops_2'])[['Social_1', 'Social_2', 'Social_3', 'Price']].mean()

,Social_1,Social_2,Social_3,Price
Shops_2,,,,
A,16.230303,3431.099394,1.458182,192339.700513
B,25.447411,5524.895586,8.630954,216098.999571


квартира имеет три разные площади:

жилая площадь квартиры;
площадь квартиры;
общая площадь квартиры.
Правила и нормативы определения общей площади помещений регулируются Жилищным Кодексом РФ, в частности ст.15, п. 5, а также Инструкцией о проведении учета жилищного фонда в Российской Федерации (утв. приказом Минземстроя РФ от 04.08.1998 г. № 37).

Жилая площадь – это сумма площадей жилых комнат. Общая площадь квартиры – это сумма площадей всех комнат, составляющих данную квартиру, в том числе подсобных помещений, кроме лоджий, балконов, веранд и террас. К подсобным помещениям относятся площади кухонь, коридоров, ванных, санузлов, встроенных шкафов, кладовых, а также площадь, занятая внутриквартирной лестницей. Ну и площадь квартиры – это сумма площадей всех помещений, составляющих квартиру, в том числе балконы, лоджии и террасы, площадь которых посчитана с определенным коэффициентом. Коэффициенты такие: для лоджий - 0,5, для балконов и террас - 0,3, для веранд и холодных кладовых - 1,0.

Категория магазинов: вероятно, shops_1 это количество близлежащих магазинов

In [64]:
data.groupby(['Shops_2'])[['Shops_1', 'Price']].mean()

,Shops_1,Price
Shops_2,,
A,2.002424,192339.700513
B,4.431717,216098.999571


In [65]:
#['DistrictId', 'Square', 'LifeSquare', 'KitchenSquare',
#       'Ecology_1', 'Ecology_2', 'EcoBA',
#       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
#       'Helthcare_2', 'Shops_1', 'Shops_2', 'Price'

In [66]:
data['SocialS'] = (data['Social_1'] + data['Social_3'])

In [67]:
#data.groupby(['DistrictId'])[['Social_1', 'Social_2', 'Social_3', 'Price', 'HouseYear']].mean()
data.groupby(['Social_1', 'Social_3'])[['Price']].agg(['mean', 'median']).head()

Price               
                            mean         median
Social_1 Social_3                              
0        0         107466.069521   96009.207535
1        0         178578.519282  175732.068626
2        0         161466.413529  155536.339047
         1         171281.077684  165627.140806
3        0         159530.416728  159530.416728

In [68]:
data.groupby(['DistrictId'])[['Social_1', 'Social_3', 'Price']].median().head()

,Social_1,Social_3,Price
DistrictId,,,
0,25.0,16.0,165963.054142
1,1.0,0.0,183663.443595
2,35.0,4.0,208539.501373
3,43.0,1.0,169094.013281
4,12.0,23.0,278639.482329


In [69]:
data.loc[:150, ['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Ecology_1', 'Ecology_2', 'EcoBA',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
       'Helthcare_2', 'Shops_1', 'Shops_2', 'Price']].head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Ecology_1,Ecology_2,EcoBA,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,35,2.0,47.981561,29.442751,6.0,0.089040,B,BB,B,33,7976,5,NaN,0,11,B,184966.930730
1,41,3.0,65.683640,40.049543,8.0,0.000070,B,BB,B,46,10309,1,240.0,1,16,B,300009.450063
2,53,2.0,44.947953,29.197612,0.0,0.049637,B,BB,B,34,7759,0,229.0,1,3,B,220925.908524
3,58,2.0,53.352981,52.731512,9.0,0.437885,B,BB,B,23,5735,3,1084.0,0,5,B,175616.227217
4,99,1.0,39.649192,23.776169,7.0,0.012339,B,BB,B,35,5776,1,2078.0,2,4,B,150226.531644


In [70]:
data['DiffSquare'] = data['Square'] - data['LifeSquare'] - data['KitchenSquare']

квартира имеет три разные площади:

жилая площадь квартиры;
площадь квартиры;
общая площадь квартиры.


Жилая площадь – это сумма площадей жилых комнат. Общая площадь квартиры – это сумма площадей всех комнат, составляющих данную квартиру, в том числе подсобных помещений, кроме лоджий, балконов, веранд и террас. К подсобным помещениям относятся площади кухонь, коридоров, ванных, санузлов, встроенных шкафов, кладовых, а также площадь, занятая внутриквартирной лестницей. Ну и площадь квартиры – это сумма площадей всех помещений, составляющих квартиру, в том числе балконы, лоджии и террасы, площадь которых посчитана с определенным коэффициентом

Квартиры с нулевой площадью кухни - скорее всего студии

In [71]:
data.loc[:, ['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'DiffSquare']].head(25)

,Rooms,Square,LifeSquare,KitchenSquare,DiffSquare
0,2.0,47.981561,29.442751,6.0,12.538811
1,3.0,65.683640,40.049543,8.0,17.634097
2,2.0,44.947953,29.197612,0.0,15.750341
3,2.0,53.352981,52.731512,9.0,-8.378531
4,1.0,39.649192,23.776169,7.0,8.873023
5,3.0,80.384479,46.683720,12.0,21.700759
6,2.0,62.254114,37.160377,7.0,18.093736
7,2.0,80.312926,NaN,0.0,NaN
8,2.0,64.511437,NaN,1.0,NaN
9,1.0,46.461409,18.915552,8.0,19.545856


In [72]:
data.groupby(['Rooms'])[['Square', 'LifeSquare', 'KitchenSquare', 'DiffSquare', 'Price']].median().head(25)

,Square,LifeSquare,KitchenSquare,DiffSquare,Price
Rooms,,,,,
0.0,65.487474,85.125471,0.0,2.212195,214937.068787
1.0,40.406590,21.663929,6.0,9.021559,154979.497993
2.0,55.841812,32.552134,6.0,11.753679,200320.754636
3.0,77.413643,46.960480,7.0,16.714846,268723.157677
4.0,98.668054,64.946548,9.0,20.399553,377524.231990
5.0,116.084861,80.458910,9.0,21.222706,455355.638212
6.0,59.414334,38.702244,6.0,14.712089,229661.964416
10.0,59.964120,37.321877,10.0,12.642244,244797.297328
19.0,42.006046,21.779288,7.0,13.226758,78364.616704


Квартира с 0 комнат явно студия - судя по стоимости, отсутствия кухни и оставшейся площади на санузел

In [73]:
data.groupby(['Rooms']).count()

,Id,DistrictId,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,...,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,EcoBA,SocialS,DiffSquare
Rooms,,,,,,,,,,,,,,,,,,,,,
0.0,8,8,8,5,8,8,8,8,8,8,...,8,8,2,8,8,8,8,8,8,5
1.0,3705,3705,3705,2827,3705,3705,3705,3705,3705,3705,...,3705,3705,1804,3705,3705,3705,3705,3705,3705,2827
2.0,3880,3880,3880,3052,3880,3880,3880,3880,3880,3880,...,3880,3880,2105,3880,3880,3880,3880,3880,3880,3052
3.0,2235,2235,2235,1842,2235,2235,2235,2235,2235,2235,...,2235,2235,1193,2235,2235,2235,2235,2235,2235,1842
4.0,150,150,150,141,150,150,150,150,150,150,...,150,150,86,150,150,150,150,150,150,141
5.0,18,18,18,16,18,18,18,18,18,18,...,18,18,10,18,18,18,18,18,18,16
6.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
10.0,2,2,2,2,2,2,2,2,2,2,...,2,2,1,2,2,2,2,2,2,2
19.0,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1


In [74]:
data = data.loc[data['Square'] > 3]
data.loc[data['Rooms'] == 0]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,EcoBA,SocialS,DiffSquare
1397,12638,27,0.0,138.427694,136.215499,0.0,4,3.0,2016,0.075424,...,3097,0,NaN,0,0,B,268394.744389,BB,11,2.212195
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,...,1892,0,NaN,0,1,B,302211.260887,BB,9,1.701236
2269,7317,27,0.0,41.790881,NaN,0.0,13,0.0,1977,0.211401,...,1892,0,NaN,0,1,B,98129.976788,BB,9,NaN
3911,770,28,0.0,49.483501,NaN,0.0,16,0.0,2015,0.118537,...,6207,1,1183.0,1,0,B,217009.338463,BB,31,NaN
4366,456,6,0.0,81.491446,NaN,0.0,4,0.0,1977,0.243205,...,1564,0,540.0,0,0,B,212864.799112,BB,5,NaN
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,...,8429,3,NaN,3,9,B,158998.110646,BB,46,10.351986
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,...,1892,0,NaN,0,1,B,219281.918007,BB,9,2.637145


In [75]:
data.loc[data['Rooms'] == 0, ['Rooms']] = 1

In [76]:
data.loc[:,['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'DiffSquare']].head()

,Rooms,Square,LifeSquare,KitchenSquare,DiffSquare
0,2.0,47.981561,29.442751,6.0,12.538811
1,3.0,65.683640,40.049543,8.0,17.634097
2,2.0,44.947953,29.197612,0.0,15.750341
3,2.0,53.352981,52.731512,9.0,-8.378531
4,1.0,39.649192,23.776169,7.0,8.873023


In [77]:
#data.groupby(['Rooms'])[['DiffSquare', 'LifeSquare', 'KitchenSquare', 'DiffSquare', 'Price']].median().head(25)
data.loc[data['DiffSquare'] < 0, ['Square']] = data['LifeSquare'] + data['KitchenSquare']

In [78]:
data.loc[data['DiffSquare'] < 0, ['DiffSquare']] = 0

In [79]:
data.groupby(['Rooms'])[['DiffSquare']].median()

,DiffSquare
Rooms,
1.0,9.024500
2.0,11.753679
3.0,16.714846
4.0,20.399553
5.0,21.222706
6.0,14.712089
10.0,12.642244
19.0,13.226758


In [80]:
data = data.loc[data['Rooms'] < 6]

In [81]:
data.groupby(['Rooms'])[['DiffSquare']].median()

,DiffSquare
Rooms,
1.0,9.024500
2.0,11.753679
3.0,16.714846
4.0,20.399553
5.0,21.222706


In [82]:
mdfs = data.groupby(['Rooms'])[['DiffSquare']].median().reset_index().rename(columns={'DiffSquare': 'MedianDiff'})

In [83]:
data = pd.merge(data, mdfs, on=['Rooms'], how='left')


In [84]:
data['LifeSquare'] = data['LifeSquare'].fillna(data['Square'] - data['MedianDiff'])

In [85]:
data['DiffSquare'] = data['Square'] - data['LifeSquare'] - data['KitchenSquare']
data.loc[data['DiffSquare'] < 0, ['DiffSquare']] = 0

In [86]:
data['SSQ'] = data['Square'] ** 2
data['LSSQ'] = data['LifeSquare'] ** 2
data['KSSQ'] = data['KitchenSquare'] ** 2
data['DSSQ'] = data['DiffSquare'] ** 2

In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9991 entries, 0 to 9990
Data columns (total 28 columns):
Id               9991 non-null int64
DistrictId       9991 non-null int64
Rooms            9991 non-null float64
Square           9991 non-null float64
LifeSquare       9991 non-null float64
KitchenSquare    9991 non-null float64
Floor            9991 non-null int64
HouseFloor       9991 non-null float64
HouseYear        9991 non-null int64
Ecology_1        9991 non-null float64
Ecology_2        9991 non-null object
Ecology_3        9991 non-null object
Social_1         9991 non-null int64
Social_2         9991 non-null int64
Social_3         9991 non-null int64
Healthcare_1     5199 non-null float64
Helthcare_2      9991 non-null int64
Shops_1          9991 non-null int64
Shops_2          9991 non-null object
Price            9991 non-null float64
EcoBA            9991 non-null object
SocialS          9991 non-null int64
DiffSquare       9991 non-null float64
MedianDiff       999

In [88]:
data = data.loc[:, ['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'HouseYear',
       'Social_1', 'Social_2', 'Social_3', 'Shops_2', 'EcoBA', 'Price', 'Ecology_1', 'DiffSquare', 'SSQ', 'LSSQ', 'KSSQ', 'DSSQ']]

In [89]:
data = pd.get_dummies(data)

In [90]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9991 entries, 0 to 9990
Data columns (total 22 columns):
Id               9991 non-null int64
DistrictId       9991 non-null int64
Rooms            9991 non-null float64
Square           9991 non-null float64
LifeSquare       9991 non-null float64
KitchenSquare    9991 non-null float64
HouseYear        9991 non-null int64
Social_1         9991 non-null int64
Social_2         9991 non-null int64
Social_3         9991 non-null int64
Price            9991 non-null float64
Ecology_1        9991 non-null float64
DiffSquare       9991 non-null float64
SSQ              9991 non-null float64
LSSQ             9991 non-null float64
KSSQ             9991 non-null float64
DSSQ             9991 non-null float64
Shops_2_A        9991 non-null uint8
Shops_2_B        9991 non-null uint8
EcoBA_AB         9991 non-null uint8
EcoBA_BA         9991 non-null uint8
EcoBA_BB         9991 non-null uint8
dtypes: float64(11), int64(6), uint8(5)
memory usage: 1.4

In [91]:
fts = ['Id', 'Rooms',
       'Social_1', 'Social_2', 'Social_3', 'Shops_2_A', 'Shops_2_B',
       'EcoBA_AB', 'EcoBA_BA', 'EcoBA_BB', 'Ecology_1', 'HouseYear',
       'SSQ', 'LSSQ', 'KSSQ', 'DSSQ']

In [92]:
train, valid = train_test_split(data, test_size = 0.3, random_state=42)

In [95]:
#max_r2_valid = 0
#max_depth_best = None
#for i in range(10, 20):
##    print('max_depth = {}'.format(i))
 #   rf = RandomForestRegressor(n_estimators=1000, max_depth=i, min_samples_leaf=2, random_state=42)
 #   rf.fit(train.loc[:, fts], train['Price'])
 #   pred = rf.predict(train.loc[:, fts])
 #   print(r2(train['Price'], pred))
 #   pred_valid = rf.predict(valid.loc[:, fts])
 #   r2_valid = r2(valid['Price'], pred_valid)
 #   print(r2_valid)
 #   print()
 #   if r2_valid > max_r2_valid:
 #       max_r2_valid = r2_valid
 #       max_depth_best = i
        
#print('max_depth_best: {}, max_r2_valid: {}'.format(max_depth_best, max_r2_valid))

In [96]:
rf = RandomForestRegressor(n_estimators=1000, max_depth=17, min_samples_leaf=2, random_state=42)
rf.fit(train.loc[:, fts], train['Price'])
pred = rf.predict(train.loc[:, fts])
print(r2(train['Price'], pred))
pred_valid = rf.predict(valid.loc[:, fts])
r2_valid = r2(valid['Price'], pred_valid)
print(r2_valid)

0.9260361905856921
0.7357681744164095
